In [1]:
import numpy as np
import xgboost as xgb
from wideboost.wrappers import wxgb

import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

(ds_train, ds_test), ds_info = tfds.load(
    'fashion_mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

for i in ds_train.batch(60000):
    a = i
    break
    
for i in ds_test.batch(60000):
    b = i
    break

In [2]:
xtrain = a[0].numpy().reshape([-1,28*28])
ytrain = a[1].numpy()

xtest = b[0].numpy().reshape([-1,28*28])
ytest = b[1].numpy()

dtrain = xgb.DMatrix(xtrain,label=ytrain)
dtest = xgb.DMatrix(xtest,label=ytest)

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    wbst = wxgb.train(param, dtrain, 20,watchlist,evals_result=ed1_results)
    
    output = min(ed1_results['test']['merror'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['I']),
    'extra_dims': hp.choice('extra_dims',[0]),
    'objective': hp.choice('objective',['multi:softmax']),
    'eval_metric':hp.choice('eval_metric',[['merror']]),
    'num_class':hp.choice('num_class',[10]),
    'eta': hp.loguniform('eta', -7, 0),
    'max_depth' : hp.choice('max_depth',range(1,11)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
    'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
    'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
    'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
}

best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'alpha': 1.664125455150004, 'btype': 'I', 'colsample_bylevel': 0.9120203363393908, 'colsample_bytree': 0.8548455795744943, 'eta': 0.0011948592066049204, 'eval_metric': ('merror',), 'extra_dims': 0, 'gamma': 0.22451342312464176, 'lambda': 0.4237210073245826, 'max_depth': 9, 'min_child_weight': 1.8790847892731356e-06, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.5110884741045645}
Overwriting param `num_class`                          
Overwriting param `objective` while setting `obj` in train.
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                
Setting param `disable_default_eval_metric` to 1.      
[14:46:26] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip throu

[0]	train-merror:0.23162	test-merror:0.24670                           

[1]	train-merror:0.20462	test-merror:0.21970                           

[2]	train-merror:0.19742	test-merror:0.21400                           

[3]	train-merror:0.19433	test-merror:0.21050                           

[4]	train-merror:0.19290	test-merror:0.20790                           

[5]	train-merror:0.19098	test-merror:0.20740                           

[6]	train-merror:0.19218	test-merror:0.20670                           

[7]	train-merror:0.19063	test-merror:0.20630                           

[8]	train-merror:0.18928	test-merror:0.20380                           

[9]	train-merror:0.19008	test-merror:0.20600                           

[10]	train-merror:0.18963	test-merror:0.20540                          

[11]	train-merror:0.18845	test-merror:0.20370                          

[12]	train-merror:0.18918	test-merror:0.20410                          

[13]	train-merror:0.18883	test-merror:0.20560      

[11]	train-merror:0.00688	test-merror:0.12750                           

[12]	train-merror:0.00485	test-merror:0.12690                           

[13]	train-merror:0.00368	test-merror:0.12540                           

[14]	train-merror:0.00202	test-merror:0.12480                           

[15]	train-merror:0.00120	test-merror:0.12440                           

[16]	train-merror:0.00060	test-merror:0.12400                           

[17]	train-merror:0.00037	test-merror:0.12300                           

[18]	train-merror:0.00020	test-merror:0.12190                           

[19]	train-merror:0.00015	test-merror:0.12150                           

NEW BEST VALUE!                                                         
{'alpha': 0, 'btype': 'I', 'colsample_bylevel': 0.8866624350527526, 'colsample_bytree': 0.8300282042640623, 'eta': 0.28657736588901156, 'eval_metric': ('merror',), 'extra_dims': 0, 'gamma': 0, 'lambda': 7.764048092238905e-06, 'max_depth': 6, 'min_child_weight':

[18]	train-merror:0.12822	test-merror:0.15750                              

[19]	train-merror:0.12830	test-merror:0.15660                              

{'alpha': 3.1237603403662694e-05, 'btype': 'I', 'colsample_bylevel': 0.6462694697036122, 'colsample_bytree': 0.5794198814601509, 'eta': 0.27837855780365156, 'eval_metric': ('merror',), 'extra_dims': 0, 'gamma': 0.14686207315435582, 'lambda': 1.3977826865287817, 'max_depth': 9, 'min_child_weight': 5.5474324574295e-06, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.6188824669404618}
Overwriting param `num_class`                                              
Overwriting param `objective` while setting `obj` in train.                
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                                    
Setting param `disable_default_eval_metric` to 1.                          
[15:56:17] WARNING: /Users/travis/build/dmlc/xgboost/

{'alpha': 6.85101275389367e-05, 'btype': 'I', 'colsample_bylevel': 0.8200813380392168, 'colsample_bytree': 0.6885526608818915, 'eta': 0.02229460499608561, 'eval_metric': ('merror',), 'extra_dims': 0, 'gamma': 0.014492606515608124, 'lambda': 7.261275169006666, 'max_depth': 1, 'min_child_weight': 4.5905211403365e-07, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.7229832690770742}
Overwriting param `num_class`                                              
Overwriting param `objective` while setting `obj` in train.                
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                                    
Setting param `disable_default_eval_metric` to 1.                          
[16:09:43] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480:   
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  

...

[0]	train-merror:0.28720	test-merror:0.29570                               

[1]	train-merror:0.24600	test-merror:0.25970                               

[2]	train-merror:0.23100	test-merror:0.24490                               

[3]	train-merror:0.22392	test-merror:0.23370                               

[4]	train-merror:0.21867	test-merror:0.23370                               

[5]	train-merror:0.21658	test-merror:0.23240                               

[6]	train-merror:0.21545	test-merror:0.22950                               

[7]	train-merror:0.21295	test-merror:0.22720                               

[8]	train-merror:0.20797	test-merror:0.22340                               

[9]	train-merror:0.20728	test-merror:0.22210                               

[10]	train-merror:0.20387	test-merror:0.21770                              

[11]	train-merror:0.20208	test-merror:0.21700                              

[12]	train-merror:0.20118	test-merror:0.21570                              


[3]	train-merror:0.19597	test-merror:0.23990                              

[4]	train-merror:0.17597	test-merror:0.22990                              

[5]	train-merror:0.17147	test-merror:0.22450                              

[6]	train-merror:0.17652	test-merror:0.23100                              

[7]	train-merror:0.17535	test-merror:0.23180                              

[8]	train-merror:0.18060	test-merror:0.23570                              

[9]	train-merror:0.17573	test-merror:0.23170                              

[10]	train-merror:0.17265	test-merror:0.22920                             

[11]	train-merror:0.17660	test-merror:0.23010                             

[12]	train-merror:0.17828	test-merror:0.23450                             

[13]	train-merror:0.18055	test-merror:0.23590                             

[14]	train-merror:0.17763	test-merror:0.23410                             

[15]	train-merror:0.20133	test-merror:0.25310                             

[16]	train-m

[8]	train-merror:0.08307	test-merror:0.13900                               

[9]	train-merror:0.07870	test-merror:0.13570                               

[10]	train-merror:0.07498	test-merror:0.13580                              

[11]	train-merror:0.07123	test-merror:0.13420                              

[12]	train-merror:0.06785	test-merror:0.13270                              

[13]	train-merror:0.06387	test-merror:0.13080                              

[14]	train-merror:0.06062	test-merror:0.13010                              

[15]	train-merror:0.05702	test-merror:0.12870                              

[16]	train-merror:0.05320	test-merror:0.12680                              

[17]	train-merror:0.04978	test-merror:0.12580                              

[18]	train-merror:0.04603	test-merror:0.12370                              

[19]	train-merror:0.04320	test-merror:0.12310                              

{'alpha': 0.0010474002277081685, 'btype': 'I', 'colsample_bylevel': 0.936305

[13]	train-merror:0.00120	test-merror:0.13950                            

[14]	train-merror:0.00097	test-merror:0.14020                            

[15]	train-merror:0.00070	test-merror:0.13830                            

[16]	train-merror:0.00087	test-merror:0.13750                            

[17]	train-merror:0.00088	test-merror:0.13750                            

[18]	train-merror:0.00130	test-merror:0.13750                            

[19]	train-merror:0.00130	test-merror:0.13710                            

{'alpha': 0.004663977394087013, 'btype': 'I', 'colsample_bylevel': 0.7492681431468863, 'colsample_bytree': 0.8797861386577468, 'eta': 0.003695457645895827, 'eval_metric': ('merror',), 'extra_dims': 0, 'gamma': 0.09511917158471189, 'lambda': 0.027215337734043107, 'max_depth': 6, 'min_child_weight': 2.791725657751504e-05, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.8093760420245351}
Overwriting param `num_class`                                            

[19]	train-merror:0.06752	test-merror:0.12690                            

{'alpha': 1.1231337414197868e-06, 'btype': 'I', 'colsample_bylevel': 0.6045834814574506, 'colsample_bytree': 0.9886325180730742, 'eta': 0.20110021743926346, 'eval_metric': ('merror',), 'extra_dims': 0, 'gamma': 7.28774891156927e-05, 'lambda': 1.173591244995448, 'max_depth': 10, 'min_child_weight': 1.1438581636900605e-07, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.9295605739081987}
Overwriting param `num_class`                                            
Overwriting param `objective` while setting `obj` in train.              
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                                  
Setting param `disable_default_eval_metric` to 1.                        
[20:53:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be ac

In [4]:
print(best_val)
print(space_eval(spc, best))

0.1159
{'alpha': 1.1231337414197868e-06, 'btype': 'I', 'colsample_bylevel': 0.6045834814574506, 'colsample_bytree': 0.9886325180730742, 'eta': 0.20110021743926346, 'eval_metric': ('merror',), 'extra_dims': 0, 'gamma': 7.28774891156927e-05, 'lambda': 1.173591244995448, 'max_depth': 10, 'min_child_weight': 1.1438581636900605e-07, 'num_class': 10, 'objective': 'multi:softmax', 'subsample': 0.9295605739081987}
